In [1]:
import gzip
from collections import defaultdict
from sklearn import linear_model
import csv
import pandas as pd
import numpy as np
import sklearn
import random

from sklearn.model_selection import train_test_split

def readJSON(path):
    null = None
    
    for l in gzip.open(path, 'rt'):
        yield eval(l)

        



In [2]:
data = list(readJSON("renttherunway_final_data.json.gz"))
# data = list(readJSON("modcloth_final_data.json.gz"))

In [3]:
data = pd.DataFrame(data)
len(data)

192544

In [4]:
# Number of rows w/ missing entries
data.isna().any(axis=1).sum()

46163

In [5]:
# Number of missing entries
data.isna().sum().sum()

64759

In [6]:
data = data.dropna(subset=["fit", "bust size", "weight", "rented for", "body type", "category", "height", "size", "age"])

In [7]:
len(data)

146431

In [8]:
192544-146381

46163

In [9]:
np.unique(data["category"])

array(['ballgown', 'blazer', 'blouse', 'blouson', 'bomber', 'buttondown',
       'caftan', 'cami', 'cape', 'cardigan', 'coat', 'combo', 'crewneck',
       'culotte', 'culottes', 'down', 'dress', 'duster', 'for', 'frock',
       'gown', 'henley', 'hoodie', 'jacket', 'jeans', 'jogger',
       'jumpsuit', 'kaftan', 'kimono', 'knit', 'legging', 'leggings',
       'maxi', 'midi', 'mini', 'overalls', 'overcoat', 'pant', 'pants',
       'parka', 'peacoat', 'poncho', 'print', 'pullover', 'romper',
       'sheath', 'shift', 'shirt', 'shirtdress', 'skirt', 'skirts',
       'skort', 'suit', 'sweater', 'sweatershirt', 'sweatpants',
       'sweatshirt', 't-shirt', 'tank', 'tee', 'tight', 'top', 'trench',
       'trouser', 'trousers', 'tunic', 'turtleneck', 'vest'], dtype=object)

In [10]:
data.head()

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
0,fit,420272,34d,2260466,137lbs,10,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,"5' 8""",14,28,"April 20, 2016"
1,fit,273551,34b,153475,132lbs,10,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,"5' 6""",12,36,"June 18, 2013"
3,fit,909926,34c,126335,135lbs,8,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,"5' 5""",8,34,"February 12, 2014"
4,fit,151944,34b,616682,145lbs,10,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,"5' 9""",12,27,"September 26, 2016"
5,fit,734848,32b,364092,138lbs,8,date,Didn't actually wear it. It fit perfectly. The...,athletic,Traditional with a touch a sass,dress,"5' 8""",8,45,"April 30, 2016"


In [11]:
def standardize_height(str_in):
    if "\'" in str_in:
        str_in = str_in.replace('"', "")
        str_in = str_in.split("\' ")
        str_in = int(str_in[0])*12 + int(str_in[1])
        
        output = str_in
    
    return output

In [12]:
data["height"] = data["height"].apply(standardize_height)

In [13]:
def standardize_weight(str_in):
    if "lbs" in str_in:
        str_in = str_in.replace("lbs", "")
        output = int(str_in)
    
    return output

In [14]:
data["weight"] = data["weight"].apply(standardize_weight)

In [15]:
def standardize_fit(str_in):
    if "small" in str_in:
        output = 0
    elif "fit" in str_in:
        output = 1
    elif "large" in str_in:
        output = 2
    return output

In [16]:
data["fit"] = data["fit"].apply(standardize_fit)

In [17]:
def standardize_size(str_in):
    output = int(str_in)
    return output

In [18]:
data["size"] = data["size"].apply(standardize_size)

In [19]:
def standardize_age(str_in):
    output = int(str_in)
    return output

In [20]:
data["age"] = data["age"].apply(standardize_age)

In [21]:
np.unique(data["body type"])

array(['apple', 'athletic', 'full bust', 'hourglass', 'pear', 'petite',
       'straight & narrow'], dtype=object)

In [22]:
def standardize_body(str_in):
    if "apple" in str_in:
        output = 0
    elif "athletic" in str_in:
        output = 1
    elif "full bust" in str_in:
        output = 2
    elif "hourglass" in str_in:
        output = 3
    elif "pear" in str_in:
        output = 4
    elif "petite" in str_in:
        output = 5
    elif "straight & narrow" in str_in:
        output = 6
    return output

In [23]:
data["body type"] = data["body type"].apply(standardize_body)

In [24]:
clothing_dict = dict(enumerate(np.unique(data["category"])))
clothing_dict = dict((v,k) for k,v in clothing_dict.items())

In [25]:
def standardize_clothing(str_in):
    if str_in in list(clothing_dict.keys()):
        output = clothing_dict[str_in]
    
    return output

In [26]:
data["category"] = data["category"].apply(standardize_clothing)

In [27]:
np.unique(data["bust size"])

array(['28a', '28aa', '28b', '28c', '28d', '28dd', '28ddd/e', '28f',
       '28g', '28h', '30a', '30aa', '30b', '30c', '30d', '30dd',
       '30ddd/e', '30f', '30g', '30h', '30i', '32a', '32aa', '32b', '32c',
       '32d', '32d+', '32dd', '32ddd/e', '32f', '32g', '32h', '32i',
       '32j', '34a', '34aa', '34b', '34c', '34d', '34d+', '34dd',
       '34ddd/e', '34f', '34g', '34h', '34i', '34j', '36a', '36aa', '36b',
       '36c', '36d', '36d+', '36dd', '36ddd/e', '36f', '36g', '36h',
       '36i', '36j', '38a', '38aa', '38b', '38c', '38d', '38d+', '38dd',
       '38ddd/e', '38f', '38g', '38h', '38i', '38j', '40b', '40c', '40d',
       '40dd', '40ddd/e', '40f', '40g', '40h', '40j', '42b', '42c', '42d',
       '42dd', '42ddd/e', '42f', '42g', '42j', '44b', '44c', '44d',
       '44dd', '44ddd/e', '44f', '44g', '44h', '46ddd/e', '48d', '48dd'],
      dtype=object)

In [28]:
def standardize_bust(str_in):
    output = str_in[:2]
    output += "-"
    
    if str_in[2:] == "a":
        output += "0"
    elif str_in[2:] == "aa":
        output += "1"
    elif str_in[2:] == "b":
        output += "2"
    elif str_in[2:] == "c":
        output += "3"
    elif str_in[2:] == "d":
        output += "4"
    elif str_in[2:] == "e":
        output += "5"
    elif str_in[2:] == "d+":
        output += "6"
    elif str_in[2:] == "ddd/e":
        output += "7"
    elif str_in[2:] == "f":
        output += "8"
    elif str_in[2:] == "g":
        output += "9"
    elif str_in[2:] == "h":
        output += "10"
    elif str_in[2:] == "i":
        output += "11"
    
    return output

In [29]:
data["bust size"] = data["bust size"].apply(standardize_bust)

In [30]:
np.unique(data["rented for"])

array(['date', 'everyday', 'formal affair', 'other', 'party',
       'party: cocktail', 'vacation', 'wedding', 'work'], dtype=object)

In [31]:
def standardize_rent(str_in):
    if "date" in str_in:
        output = 0
    elif "everyday" in str_in:
        output = 1
    elif "formal affair" in str_in:
        output = 2
    elif "other" in str_in:
        output = 3
    elif "party" in str_in:
        output = 4
    elif "party: cocktail" in str_in:
        output = 5
    elif "vacation" in str_in:
        output = 6
    elif "wedding" in str_in:
        output = 7
    elif "work" in str_in:
        output = 8
    return output

In [32]:
data["rented for"] = data["rented for"].apply(standardize_rent)

In [33]:
data.head()

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
0,1,420272,34-4,2260466,137,10,6,An adorable romper! Belt and zipper were a lit...,3,So many compliments!,44,68,14,28,"April 20, 2016"
1,1,273551,34-2,153475,132,10,3,I rented this dress for a photo shoot. The the...,6,I felt so glamourous!!!,20,66,12,36,"June 18, 2013"
3,1,909926,34-3,126335,135,8,2,I rented this for my company's black tie award...,4,Dress arrived on time and in perfect condition.,16,65,8,34,"February 12, 2014"
4,1,151944,34-2,616682,145,10,7,I have always been petite in my upper body and...,1,Was in love with this dress !!!,20,69,12,27,"September 26, 2016"
5,1,734848,32-2,364092,138,8,0,Didn't actually wear it. It fit perfectly. The...,1,Traditional with a touch a sass,16,68,8,45,"April 30, 2016"


In [34]:
data = data.to_dict('records')

In [35]:
data

[{'fit': 1,
  'user_id': '420272',
  'bust size': '34-4',
  'item_id': '2260466',
  'weight': 137,
  'rating': '10',
  'rented for': 6,
  'review_text': "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.",
  'body type': 3,
  'review_summary': 'So many compliments!',
  'category': 44,
  'height': 68,
  'size': 14,
  'age': 28,
  'review_date': 'April 20, 2016'},
 {'fit': 1,
  'user_id': '273551',
  'bust size': '34-2',
  'item_id': '153475',
  'weight': 132,
  'rating': '10',
  'rented for': 3,
  'review_text': 'I rented this dress for a photo shoot. The theme was "Hollywood Glam and Big Beautiful Hats". The dress was very comfortable and easy to move around in. It is definitely on my list to rent again for another formal event. ',
  'body type': 6,
  'review_summary': 'I felt so glamourous!!!',
  'category': 20,
  '

In [36]:
mean_weight = np.mean([i["weight"] for i in data])
std_weight = np.std([i["weight"] for i in data])

mean_height = np.mean([i["height"] for i in data])
std_height = np.std([i["height"] for i in data])

mean_size = np.mean([i["size"] for i in data])
std_size = np.std([i["size"] for i in data])

In [37]:
def feat(datum):
    body = [0]*7
    body[datum["body type"]] = 1
    body = body[:6]
    
    category = [0]*69
    category[datum["category"]] = 1
    category = category[:68]
    
    # bust_list is a list where [a, b] a is the cup size, b is the bust size?
    bust_list = datum["bust size"].split("-")
    
    bust = [0]*12
    if bust_list[1] != "":
        bust[int(bust_list[1])] = 1
        
    rent = [0]*9
    rent[datum["rented for"]] = 1
    rent = rent[:8]
    
#     return [1] + [datum["height"]] + [datum["weight"]] + [datum["size"]] + [datum["age"]] + [int(bust_list[0])] + body + bust + category + rent
    return [1] + [(datum["height"] - mean_height)/std_height] + [(datum["weight"] - mean_weight)/std_weight] + [(datum["size"] - mean_size)/std_size] + [datum["age"]] + [int(bust_list[0])] + body + bust + category + rent

In [38]:
feat(data[0])

[1,
 1.029354524917305,
 -0.009668215752023519,
 0.32739412804262813,
 28,
 34,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0]

In [39]:
# Equalize number of small, fit, large labels in the data.
small = [i for i in data if i["fit"]==0]
fit = [i for i in data if i["fit"]==1]
large = [i for i in data if i["fit"]==2]


random.shuffle(fit)
data = small+fit[:19623]+large

In [40]:
len(data)

58284

In [41]:
X = [feat(d) for d in data]
y = [d["fit"] for d in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [42]:
model = sklearn.linear_model.LogisticRegression(solver="saga", fit_intercept=True, multi_class='multinomial', C=0.1)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [43]:
print("Model Accuracy on validation set:")
sum([predictions[i]==y_test[i] for i in range(len(y_test))])/len(y_test)

Model Accuracy on validation set:


0.47979754653856055

In [44]:
# Accuracy of predictions when actual label was small
sum([predictions[i]==y_test[i] for i in range(len(predictions)) if y_test[i] == 0])/len([i for i in y_test if i == 0])

0.5922157211905368

In [45]:
# Accuracy of predictions when actual label was fit
sum([predictions[i]==y_test[i] for i in range(len(predictions)) if y_test[i] == 1])/len([i for i in y_test if i == 1])

0.29395181957970273

In [46]:
# Accuracy of predictions when actual label was large
sum([predictions[i]==y_test[i] for i in range(len(predictions)) if y_test[i] == 2])/len([i for i in y_test if i == 2])

0.5538702928870293

In [47]:
model.predict([feat(fit[0])])

array([2])

In [48]:
fit[0]

{'fit': 1,
 'user_id': '821540',
 'bust size': '36-3',
 'item_id': '1238932',
 'weight': 150,
 'rating': '8',
 'rented for': 7,
 'review_text': 'Pretty royal blue dress.  Fit comfortably, with a cute cutout.  Safe to wear a bra with.  ',
 'body type': 3,
 'review_summary': 'Comfortable, cute cut out',
 'category': 16,
 'height': 68,
 'size': 12,
 'age': 34,
 'review_date': 'April 24, 2017'}

In [49]:
large[0]

{'fit': 2,
 'user_id': '533900',
 'bust size': '34-2',
 'item_id': '130259',
 'weight': 135,
 'rating': '8',
 'rented for': 7,
 'review_text': "This dress was absolutely gorgeous and I received so many compliments!  I wore it to a black tie NYE wedding, and even though it was short, I felt like the fabric, fit, and the way it sparkled made it fit in just fine with the other formal gowns.  I usually wear a 6 or an 8 in dresses.  I ordered a 4 and a 6, and true to other reviewers' comments, the 4 fit me very well.  The chest area was just a tad loose, but that typically happens when I try on dresses...nothing a more padded bra couldn't fix!  The cut and fit were very figure flattering.  By the end of the evening I experience a little chafing under my arms from the sequins, but that was after 5 hours and it was very minor.  Thank you RTR for a great first experience, I'll definitely be back!",
 'body type': 4,
 'review_summary': "Stunning dress, perfect for a New Year's Eve wedding!",
 'c

In [50]:
small[0]

{'fit': 0,
 'user_id': '185966',
 'bust size': '34-2',
 'item_id': '1077123',
 'weight': 135,
 'rating': '8',
 'rented for': 4,
 'review_text': "The dress arrived with a small hole in the beading on the front but wasn't too noticeable. Glad I was able to get two sizes because the 4 was a little tight and would've made for an uncomfortable night of dancing! ",
 'body type': 1,
 'review_summary': "It was fun to wear a dress I wouldn't normally buy! ",
 'category': 16,
 'height': 63,
 'size': 12,
 'age': 33,
 'review_date': 'January 2, 2018'}